In [6]:
import pandas as pd
import os
from pypdf import PdfReader
import numpy as np
#os.listdir("data")
import random
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [7]:
import os, sys

# Add project root to Python path for imports
# Get the absolute path to the project root
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Try multiple methods to find project root
project_root = None

# Method 1: If we're in notebooks directory, go up one level
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
    print(f"Method 1: Found project root (from notebooks dir): {project_root}")
else:
    # Method 2: Check if helpers directory exists in current or parent
    if os.path.exists(os.path.join(current_dir, 'helpers')):
        project_root = current_dir
        print(f"Method 2: Found project root (helpers in current dir): {project_root}")
    elif os.path.exists(os.path.join(os.path.dirname(current_dir), 'helpers')):
        project_root = os.path.dirname(current_dir)
        print(f"Method 3: Found project root (helpers in parent dir): {project_root}")
    else:
        # Method 4: Try going up from current directory
        parent_dir = os.path.dirname(current_dir)
        if os.path.exists(os.path.join(parent_dir, 'helpers')):
            project_root = parent_dir
            print(f"Method 4: Found project root (going up one level): {project_root}")
        else:
            # Method 5: Use absolute path - hardcode the known project path
            # This is a fallback if all else fails
            known_project_root = "/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13"
            if os.path.exists(os.path.join(known_project_root, 'helpers')):
                project_root = known_project_root
                print(f"Method 5: Using known absolute path: {project_root}")

# Verify helpers directory exists and add to path
if project_root and os.path.exists(os.path.join(project_root, 'helpers')):
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    print(f"✅ Added project root to path: {project_root}")
    print(f"✅ Verified helpers directory exists: {os.path.exists(os.path.join(project_root, 'helpers', 'SkillsEmbeddings.py'))}")
    print(f"✅ You can now import: from helpers.SkillsEmbeddings import ...")
else:
    print(f"⚠️ Warning: Could not find helpers directory. Project root: {project_root}")
    print(f"⚠️ Current sys.path: {sys.path[:3]}")  # Show first 3 entries

Current working directory: /Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/notebooks
Method 1: Found project root (from notebooks dir): /Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13
✅ Added project root to path: /Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13
✅ Verified helpers directory exists: True
✅ You can now import: from helpers.SkillsEmbeddings import ...


In [8]:
#!pip install ollama


In [9]:
#! pip install nltk

In [10]:
#!pip install sentence-transformers
#!pip install torch
#!pip install numpy
#!pip install scipy
#!pip install pandas
#!pip install pyyaml
#!pip install streamlit
#!pip install pymupdf
#!pip install pdfplumber
#!pip install anthropic
#!pip install openai
#!pip install google-generativeai
#!pip install watchdog
#!pip install altair

In [11]:
#!pip install -U sentence-transformers
#!pip install pypdf
#!pip install transformers
#!pip install pandas


In [12]:
#from sentence_transformers import SentenceTransformer
#from SkillsEmbeddings import SkillEmbeddingModel, save_skills_emb

#model = SentenceTransformer("BAAI/bge-m3")
#skill_model = SkillEmbeddingModel(model)

#save_skills_emb(
    #input_csv="skills-finder-hackathon-hec-x-sbi/skillscleaned.csv",
    #output_npz="skills-finder-hackathon-hec-x-sbi/skills_encoded.npz",
    #model=skill_model)

In [ ]:
from sentence_transformers import SentenceTransformer
from helpers.SkillsEmbeddings import SkillEmbeddingModel, save_skills_emb
import os

# Get project root (should be set in cell 1)
# If not, detect it here
if 'project_root' not in globals():
    current_dir = os.getcwd()
    if os.path.basename(current_dir) == 'notebooks':
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir
else:
    # Use the project_root from cell 1
    pass


model = SentenceTransformer("BAAI/bge-m3")

skill_model = SkillEmbeddingModel(model)


# Use absolute paths based on project root
input_csv = os.path.join(project_root, "skills-finder-hackathon-hec-x-sbi", "skillscleaned.csv")
output_npz = os.path.join(project_root, "skills-finder-hackathon-hec-x-sbi", "skills_encoded.npz")

print(f"Input CSV: {input_csv}")
print(f"Output NPZ: {output_npz}")

save_skills_emb(
    input_csv=input_csv,
    output_npz=output_npz,
    model=skill_model
)

print("New embeddings saved")

🔄 Loading BGE-m3 model...
🔄 Initializing embedding wrapper...
🔄 Regenerating skills_encoded.npz with BGE-m3 embeddings...
📁 Input CSV: /Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/skills-finder-hackathon-hec-x-sbi/skillscleaned.csv
📁 Output NPZ: /Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/skills-finder-hackathon-hec-x-sbi/skills_encoded.npz
✅ DONE! New embeddings saved successfully.


# extracting pdf text for encoding 

In [16]:
from pypdf import PdfReader
def extract_text(filename: str):
    reader = PdfReader(filename)
    complete_text = ""
    # getting a specific page from the pdf file
    for page in reader.pages:
        complete_text += f"\n{page.extract_text()}"
    return complete_text

In [22]:
# Get project root if not already set
if 'project_root' not in globals():
    current_dir = os.getcwd()
    if os.path.basename(current_dir) == 'notebooks':
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

data_dir = os.path.join(project_root, "data")
print(os.listdir(data_dir))
# .DS_Store

['HCK_HEC_LANG.csv', '.DS_Store', 'HCK_HEC_XP.csv', 'HCK_HEC_SKILLS.csv', 'HCK_HEC_USER.csv', 'HCK_HEC_STAFFING.csv', 'processed', 'raw']


# loading all file at the same time not so useful 

# cleaning the lang dataset 
it contains the user IDs, the langauge and the skill level
there are some NAs values that we replace by -1 (need to check why we did that in the first place)  

In [144]:
csvfile = pd.read_csv("/Users/inesbenhamza/Downloads/skills-finder-hackathon-hec-x-sbi/HCK_HEC_LANG.csv")


In [145]:
# number of unique id 
csvfile['USER_ID'].nunique()

112

In [146]:
print(csvfile[csvfile['USER_ID']==2433141])
csvfile['LANGUAGE_SKILL_LVL'].isna().sum()

     USER_ID LANGUAGE_SKILL_DSC  LANGUAGE_SKILL_LVL
38   2433141             French               100.0
102  2433141            English               100.0


np.int64(8)

In [27]:
csvfile['LANGUAGE_SKILL_LVL'] = csvfile['LANGUAGE_SKILL_LVL'].fillna(-1) # to handle missing values we fill them with -1

In [28]:
csvfile['LANGUAGE_SKILL_LVL'].isna().sum()


np.int64(0)

In [ ]:
# checking if there a re duplicated rows 
print(csvfile.duplicated().sum())

0


In [30]:
#idx = csvfile.groupby(['USER_ID', 'LANGUAGE_SKILL_DSC'])['LANGUAGE_SKILL_LVL'].idxmax()

#idx = idx[idx.notna()]

#df_filtered = csvfile.loc[idx]

#df_filtered = df_filtered.reset_index(drop=True)


In [31]:
df_filtered = csvfile.loc[csvfile.groupby(['USER_ID', 'LANGUAGE_SKILL_DSC'])['LANGUAGE_SKILL_LVL'].idxmax()]
 
# Reset index if needed
df_filtered = df_filtered.reset_index(drop=True)

print(df_filtered.head(10))

   USER_ID LANGUAGE_SKILL_DSC  LANGUAGE_SKILL_LVL
0  2432194            English               100.0
1  2432194             French               100.0
2  2433083            English                80.0
3  2433083             French               100.0
4  2433092            English                80.0
5  2433092             French               100.0
6  2433099            English                80.0
7  2433099             French               100.0
8  2433109            English                80.0
9  2433109             French               100.0


In [32]:
df_filtered[df_filtered['USER_ID'] == 2664664]

,USER_ID,LANGUAGE_SKILL_DSC,LANGUAGE_SKILL_LVL
170,2664664,English,-1.0
171,2664664,French,-1.0


In [33]:
print (df_filtered.duplicated().sum())

0


# storing lang clean into a csv 

In [34]:
df_filtered.to_csv("langcleaned.csv", index=False)


create a consultant_profile_df by merging all csv ( should we creat col skills 1, skills 2 etc ? or skilss : skills1,skills2 etc)

In [ ]:
# gettign workign directory 
#os.getcwd()

In [37]:
expyears  = pd.read_csv("/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/data/HCK_HEC_USER.csv")
expyears['USER_ID'].nunique()

print(expyears.head())

   USER_ID LAST_NAME FIRST_NAME  ANNEES_XP
0  2843838     Ramos      David        1.0
1  2479537     Small       Carl        2.5
2  2533337     Evans     Carmen        2.0
3  2446382   Pittman     Brandi        2.5
4  2433124    Thomas      Julie        2.5


# USer an dtheir name + experience in year is clean 

In [38]:
print(expyears.duplicated().sum())
# print if there is user id that are duplicated 
print(expyears[expyears.duplicated(subset=['USER_ID'], keep=False)])


0
Empty DataFrame
Columns: [USER_ID, LAST_NAME, FIRST_NAME, ANNEES_XP]
Index: []


In [40]:
skillsclean = pd.read_csv("/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/data/raw/skillscleaned.csv")

skillsclean
#print(print(skillsclean[skillsclean.duplicated(subset=['USER_ID'], keep=True)]))



,USER_ID,SKILLS_DSC,DOMAIN_DSC,LEVEL_VAL
0,2433083,DBT,Data Integration,80.0
1,2433083,Microsoft Analytics Platform System (SSAS),Data Management,80.0
2,2433083,Microsoft Azure Data Factory,Data Integration,100.0
3,2433083,Microsoft Azure Data Lake Store,Data Management,90.0
4,2433083,Microsoft Azure SQL Database,Data Management,90.0
...,...,...,...,...
979,2978961,Snowflake Data Cloud,Data Management,0.0
980,2978961,Tableau Desktop and Online,Data Analytics,0.0
981,3072774,Python,Programming Language,40.0
982,3072774,R,Programming Language,40.0


In [41]:
skillsclean['DOMAIN_DSC'].unique()

array(['Data Integration', 'Data Management', 'Agile Delivery',
       'Financial Close', 'Data Analytics', 'Programming Language',
       'Data Platform', 'Data Governance', 'Data Science',
       'Financial Planning and Analysis'], dtype=object)

In [ ]:
skillsclean['USER_ID'].nunique()

93

In [43]:

skills_df = skillsclean  

# Pivot the table to create a domain-based skill dictionary per consultant
pivoted_df = skills_df.pivot_table(
    index='USER_ID',
    columns='DOMAIN_DSC',
    values='LEVEL_VAL',
    aggfunc=lambda x: dict(zip(skills_df.loc[x.index, 'SKILLS_DSC'], x))
)

# Reset index so USER_ID becomes a column
pivoted_df = pivoted_df.reset_index()

# Display result
pivoted_df

pivoted_df.to_csv("skills_pivoted.csv", index=False)

In [44]:
print(pivoted_df.head())

DOMAIN_DSC  USER_ID                                     Agile Delivery  \
0           2433083                                                NaN   
1           2433092  {'Atlassian JIRA Software': 100.0, 'Microsoft ...   
2           2433099  {'Atlassian JIRA Software': 80.0, 'Microsoft A...   
3           2433109  {'Atlassian JIRA Software': 100.0, 'Microsoft ...   
4           2433111                                                NaN   

DOMAIN_DSC                                     Data Analytics Data Governance  \
0                                                         NaN             NaN   
1                                                         NaN             NaN   
2                            {'SAP BusinessObjects BI': 20.0}             NaN   
3                                                         NaN             NaN   
4           {'Microsoft Power BI': 80.0, 'Tableau Desktop ...             NaN   

DOMAIN_DSC                                   Data Integration  \
0  

merging
with availibility 
expereince 
mission 
lang 

In [45]:
# checking the col 
pivoted_df
print(pivoted_df.columns)

Index(['USER_ID', 'Agile Delivery', 'Data Analytics', 'Data Governance',
       'Data Integration', 'Data Management', 'Data Platform', 'Data Science',
       'Financial Close', 'Financial Planning and Analysis',
       'Programming Language'],
      dtype='object', name='DOMAIN_DSC')


In [ ]:
#left merge to keep all rows from expyears 

fulldf = expyears.merge(pivoted_df, on="USER_ID", how="left")
fulldf['USER_ID'].nunique()

119

# df with user id, experience so ( USER) and SKILLS 

In [ ]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,Data Science,Financial Close,Financial Planning and Analysis,Programming Language
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Java': 70.0, 'Javascript': 20.0, 'PL/SQL': 5..."
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,NaN,{'Oracle Hyperion Financial Management (HFM)':...,"{'Applied OLAP Dodeca': 80.0, 'Oracle Cloud EP...","{'Java': 60.0, 'VBA': 100.0}"
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,"{'Anaconda Enterprise': 0.0, 'Matlab': 0.0}",NaN,{'IBM Cognos TM1': 10.0},"{'DAX': 20.0, 'Java': 20.0, 'Python': 40.0, 'S..."
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,NaN,NaN,NaN,"{'Java': 40.0, 'Python': 60.0, 'R': 40.0, 'SQL..."
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,NaN,"{'CCH Tagetik CPM Conso': 60.0, 'Oracle Hyperi...","{'CCH Tagetik CPM Planning': 80.0, 'Oracle Hyp...","{'Java': 50.0, 'Python': 40.0, 'R': 40.0, 'SQL..."
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",NaN,NaN,NaN,"{'Java': 30.0, 'Python': 70.0, 'SQL': 80.0, 'S..."


In [ ]:
availibility= pd.read_csv('/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/data/HCK_HEC_STAFFING.csv')

In [ ]:
fulldf = fulldf.merge(availibility, on="USER_ID", how="left")

In [ ]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,100
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,100
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,25,25,25
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,25,25,25,50,25,25,25
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,25,25,25,50,25,25,25,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,25,25,25
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,0,0,0
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,0,25,0,0,0,0,0,0,0,0
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,25,25,25,50,25,25,25,25


for lang and mission multiple col 

In [ ]:
langskills = pd.read_csv ('/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/langcleaned.csv')

# Group by USER_ID and build dict of LANG: LEVEL
lang_profiles = langskills.groupby("USER_ID").apply(
    lambda x: dict(zip(x["LANGUAGE_SKILL_DSC"], x["LANGUAGE_SKILL_LVL"]))
).reset_index(name="LANGUAGE_PROFILE")



/var/folders/ng/wz4hfzq14nggvhjg7kkdsts80000gn/T/ipykernel_57208/1720805847.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lang_profiles = langskills.groupby("USER_ID").apply(


In [ ]:
fulldf = fulldf.merge(lang_profiles, on="USER_ID", how="left")

fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,LANGUAGE_PROFILE
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,"{'English': 100.0, 'French': 100.0}"
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,"{'English': 90.0, 'French': 100.0}"
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,25,25,"{'English': 50.0, 'French': 100.0}"
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,25,25,50,25,25,25,"{'English': 60.0, 'French': 100.0}"
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,25,25,50,25,25,25,25,25,"{'English': 100.0, 'French': 100.0}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,25,25,"{'English': 100.0, 'French': 80.0}"
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,0,0,"{'English': 100.0, 'French': 100.0}"
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,0,0,0,0,0,0,0,0,"{'English': 80.0, 'French': 100.0}"
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,25,25,50,25,25,25,25,"{'English': 70.0, 'French': 100.0}"


In [ ]:
mission = pd.read_csv('/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/data/HCK_HEC_XP.csv')

# here we gorup mission by user so that it become 
USER_ID	MISSIONS_LIST
2843838	[mission1, mission2, mission3, …]

In [ ]:

missions_grouped = mission.groupby("USER_ID")["MISSION_DSC"].apply(list).reset_index(name="MISSIONS_LIST")


missions_grouped["MISSIONS_TEXT"] = missions_grouped["MISSIONS_LIST"].apply(lambda x: " ".join(x))

print (missions_grouped)

     USER_ID                                      MISSIONS_LIST  \
0    2432194  [Analyse de la situation actuelle et définitio...   
1    2433083  [Exhaustive data analysis, Translation of the ...   
2    2433092  [Gestion du budget., Planification de l'échéan...   
3    2433099  [Faire le suivi des activités de configuration...   
4    2433109  [Understanding of customer requirements, Analy...   
..       ...                                                ...   
109  2957009  [Collaborate with stakeholders, users, and dev...   
110  2976032  [*      Coordination with different team membe...   
111  2977311  [Increased product KPIs  by implementing a use...   
112  2978961  [Interface entre les utilisateurs finaux / par...   
113  3072774  [- Restructuration of the Centers Of Excellenc...   

                                         MISSIONS_TEXT  
0    Analyse de la situation actuelle et définition...  
1    Exhaustive data analysis Translation of the bu...  
2    Gestion du budget. 

In [ ]:
fulldf = fulldf.merge(missions_grouped, on="USER_ID", how="left")


In [ ]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,LANGUAGE_PROFILE,MISSIONS_LIST,MISSIONS_TEXT
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,"{'English': 100.0, 'French': 100.0}",[Mise en place d’un pipeline RAG pour la migra...,Mise en place d’un pipeline RAG pour la migrat...
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,"{'English': 90.0, 'French': 100.0}",[Mise en œuvre d’une application de simulation...,Mise en œuvre d’une application de simulation ...
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,"{'English': 50.0, 'French': 100.0}","[Intégration de données, Analyser et résoudre ...",Intégration de données Analyser et résoudre le...
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,50,25,25,25,"{'English': 60.0, 'French': 100.0}","[Analyser le système d'information financier, ...",Analyser le système d'information financier Co...
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,50,25,25,25,25,25,"{'English': 100.0, 'French': 100.0}",[Restitution et transmission des connaissances...,Restitution et transmission des connaissances ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,"{'English': 100.0, 'French': 80.0}","[Support aux utilisateurs finaux de niveau 2, ...",Support aux utilisateurs finaux de niveau 2 Pr...
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,"{'English': 100.0, 'French': 100.0}",[Analyses des fichiers Excel utilisés pour aut...,Analyses des fichiers Excel utilisés pour auto...
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,"{'English': 80.0, 'French': 100.0}",[**Business Analyst for the implementation of ...,**Business Analyst for the implementation of a...
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,50,25,25,25,25,"{'English': 70.0, 'French': 100.0}",[Design and implementation of data architectur...,Design and implementation of data architecture...


In [ ]:
fulldf.to_csv("full_df.csv", index=False)

In [ ]:
print(fulldf['USER_ID'].nunique())

119


NLP / Similarity : Utiliser des embeddings (e.g. SentenceTransformer) pour comparer missions passées aux responsabilités du mandat


In [4]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

def compute_similarity(text1, text2):

    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores.item()


text1 = "Created various Customized and interactive Reports and Dashboards in Power BI."
text2 = "developement de dashboard sur Power BI"
text3 = "J'aime courir"
text4 = "Mettre en place un flux de données depuis un ERP vers une database Snowflake"
text5 = "Automatiser les déploiements et les tests des pipelines via Azure DevOps (CI/CD, versioning, rollback)."
text6 = "implémenter un pipeline d’intégration de données entre un système ERP et une base de données Snowflake"
#compute_similarity(text3, text1)
#compute_similarity(text3, text2)
print(compute_similarity(text1, text6))
print(compute_similarity(text1, text3))
print(compute_similarity(text1, text2))


0.3277907371520996
0.15160588920116425
0.7804136276245117


In [5]:
print(compute_similarity(text1, text6))
print(compute_similarity(text1, text3))
print(compute_similarity(text1, text2))

0.3277907371520996
0.15160588920116425
0.7804136276245117


fine tuning ?

data engineer responsabilité 
  - "Concevoir et développer des pipelines d'ingestion de données robustes et optimisés à l'aide d'Azure Data Factory"
  - "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"
  - "Implémenter des flux de données streaming et batch via Databricks Streaming et Apache Kafka"
  - "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"
  - "Mettre en œuvre des processus de validation de la qualité des données"
  - "Automatiser les déploiements et les tests des pipelines via Azure DevOps"
  - "Documenter les solutions développées selon les normes internes"
  - "Travailler en collaboration avec les équipes Data Science, BI et IT Infrastructure"
  - "Participer à l’élaboration des principes d’architecture data et de gouvernance des données"

  need to encode the data engineer reponsabilities 

In [2]:
import pandas as pd
import os 

In [3]:
if 'project_root' not in globals():
    current_dir = os.getcwd()
    if os.path.basename(current_dir) == 'notebooks':
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

data_dir = os.path.join(project_root, "data")
print(os.listdir(data_dir))

full_df = pd.read_csv(
    os.path.join(project_root, "full_df.csv"),
    sep=',', 
    on_bad_lines='skip', 
    encoding='utf-8'
)

['HCK_HEC_LANG.csv', '.DS_Store', 'HCK_HEC_XP.csv', 'HCK_HEC_SKILLS.csv', 'HCK_HEC_USER.csv', 'HCK_HEC_STAFFING.csv', 'processed', 'raw']


In [15]:
print(full_df.loc[full_df["USER_ID"] ==2475141, "MISSIONS_TEXT"])

18    Reinforce and Develop brand awareness #FUNgeni...
Name: MISSIONS_TEXT, dtype: object


In [25]:
full_df["USER_ID"].unique()

array([ 2843838,  2479537,  2533337,  2446382,  2433124,  2817191,
        2936770,  2501258,  2433139,  2480827,  2433122,  2433125,
        2513848,  2433130,  2447746,  2433144,  2433117,  2433135,
        2475141,  2445183,  2671704,  2501164,  2451715,  2446130,
        2452725,  2977311,  2583132,  2513866,  2443989,  2700124,
        2836571,  2683916,  2978961,  2432194,  2453262,  2443199,
        3072774,  2726335,  2433140,  2503861,  2433083,  2471586,
        2578963,  2903451,  2766619,  2841644,  2708864,  2967233,
        2446029,  2693756,  2513831,  2453233,  2552838,  2445156,
        2433109,  2433142,  2437177,  2449548,  2450849,  2502347,
        2645516,  2447527,  2433099,  2433118,  2433114,  2818577,
        2634429,  2725267,  2433134,  2433120,  2664664,  2443208,
        2456839,  2445149,  2451804,  2433141,  2540340,  2708468,
        2447137,  2433111,  2444211,  2443771,  2599691,  2976032,
        2433092,  2676328,  2689959,  2551856,  2433143,  2449

In [27]:
full_df["USER_ID"].nunique()

128

fine tuning bert for paraphrase detection (are the two sentences paraphrases of each other?)

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")


positive_pairs = [
    
    ("Built dashboards in Power BI", "Créer des tableaux de bord Power BI"),
    ("Created data pipelines using ADF", "Concevoir des pipelines d’ingestion de données avec Azure Data Factory"),

    
    ("Design and develop robust and optimized data ingestion pipelines using Azure Data Factory",
     "Concevoir et développer des pipelines d'ingestion de données robustes et optimisés à l'aide d'Azure Data Factory"),

    ("Develop complex transformations using Apache Spark (Scala/Python) in Azure Databricks",
     "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),

    ("Implement streaming and batch data flows via Databricks Streaming and Apache Kafka",
     "Implémenter des flux de données streaming et batch via Databricks Streaming et Apache Kafka"),

    ("Participate in defining and implementing data models adapted to analytical needs",
     "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),

    ("Implement data quality validation processes",
     "Mettre en œuvre des processus de validation de la qualité des données"),

    ("Automate pipeline deployments and tests via Azure DevOps",
     "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),

    ("Document developed solutions according to internal standards",
     "Documenter les solutions développées selon les normes internes"),

    ("Collaborate with Data Science, BI, and IT Infrastructure teams",
     "Travailler en collaboration avec les équipes Data Science, BI et IT Infrastructure"),

    ("Contribute to the development of data architecture and data governance principles",
     "Participer à l'élaboration des principes d'architecture data et de gouvernance des données"),

    ("Facilitated Scrum ceremonies including planning and retrospectives",
     "Faciliter l’ensemble des cérémonies Scrum"),
    ("Acted as Agile coach for cross-functional team",
     "Agir en tant que coach Agile"),
    ("Helped Product Owner manage backlog and define priorities",
     "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Tracked team velocity and analyzed sprint metrics",
     "Mesurer et analyser la performance de l’équipe via des indicateurs clés"),
    ("Protected team from external distractions",
     "Protéger l’équipe des interférences externes"),

    ("Built dashboards in Power BI",
     "Développer des tableaux de bord et rapports dans Power BI"),
    ("Wrote performant SQL queries to transform marketing data",
     "Rédiger du code SQL performant"),
    ("Performed data integration from CRM and Google Analytics",
     "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),
    ("Collaborated with analysts to understand reporting needs",
     "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
    ("Designed data models aligned with business goals",
     "Concevoir des modèles de données adaptés aux objectifs métiers"),
    ("Facilitated Scrum ceremonies including planning and retrospectives",
     "Faciliter l’ensemble des cérémonies Scrum"),
    ("Acted as Agile coach for cross-functional team",
     "Agir en tant que coach Agile"),
    ("Helped Product Owner manage backlog and define priorities",
     "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Tracked team velocity and analyzed sprint metrics",
     "Mesurer et analyser la performance de l’équipe via des indicateurs clés"),
    ("Protected team from external distractions",
     "Protéger l’équipe des interférences externes"),
]

negative_pairs = [
    ("Built dashboards in Power BI", "Préparer des repas en cuisine"),
    ("Created REST APIs", "Aider les patients dans les soins"),
    ("Collaborated with ML and BI teams", "Mesurer la performance de l’équipe via des burndown charts"),
    ("Developed PySpark jobs", "Travailler avec des personas marketing pour la stratégie de contenu"),
    ("Implemented Kafka streaming", "Créer des affiches pour un événement social"),
    ("Automated deployment with Azure DevOps", "Gérer les présences dans un club de sport"),
    ("Helped define data architecture", "Traduire un document marketing en espagnol"),
    ("Documented pipelines", "Rédiger une lettre de motivation"),
    ("Assisted customers in retail", "Concevoir des modèles de données métiers"),
    ("Conducted financial audits", "Faciliter les cérémonies Scrum"),
]

train_examples = [
    InputExample(texts=[text1, text2], label=1.0) for text1, text2 in positive_pairs
] + [
    InputExample(texts=[text1, text2], label=0.0) for text1, text2 in negative_pairs
]

# Improved training setup
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)  # Increased from 1

# Use ContrastiveLoss instead of CosineSimilarityLoss for binary labels
# ContrastiveLoss is designed for positive/negative pairs with margin-based learning
train_loss = losses.ContrastiveLoss(
    model,
    distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE,
    margin=0.5  # Margin between positive and negative pairs
)

print(f"Training with {len(train_examples)} examples")
print(f"Positive pairs: {len(positive_pairs)}, Negative pairs: {len(negative_pairs)}")
print(f"Using ContrastiveLoss with margin=0.5")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,  # Increased from 3
    warmup_steps=100,  # Increased from 10
    optimizer_params={'lr': 2e-5},  # Explicit learning rate
    show_progress_bar=True
)

model.save("modelfinetuned")
print("✅ Model saved to modelfinetuned")


Training with 36 examples
Positive pairs: 26, Negative pairs: 10
Using ContrastiveLoss with margin=0.5


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/inesbenhamza/Downloads/SBI - Équipe 1 (Team 1) 13/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


✅ Model saved to modelfinetuned


In [ ]:
#import sys
#import subprocess

#subprocess.check_call([
    #sys.executable, "-m", "pip", "install", "accelerate>=0.26.0"
#])


autre
essaie 

In [5]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer("modelfinetuned")
positive_pairs = [
    ("Built dashboards in Power BI", "Créer des tableaux de bord Power BI"),
    ("Created data pipelines using ADF", "Concevoir des pipelines d’ingestion de données avec Azure Data Factory"),
    ("Develop complex transformations using Apache Spark", "Développer des transformations complexes avec Apache Spark"),
    ("Developed PySpark jobs on Databricks", "Créer des transformations complexes avec Spark dans Azure Databricks"),
    ("Built CI/CD workflows using Azure DevOps", "Automatiser les déploiements via Azure DevOps (CI/CD)"),
    ("Designed a lakehouse architecture", "Participer à l’élaboration de l’architecture data"),
    ("Monitored and maintained ADF pipelines in production", 
     "Assurer la qualité et la fiabilité des pipelines dans Azure Data Factory"),
    ("Worked closely with stakeholders to define KPIs and dashboards", 
     "Collaborer avec les analystes pour concevoir des tableaux de bord métier"),
    ("Wrote optimized SQL for analytics reporting", 
     "Rédiger du code SQL performant pour l’analyse de données"),
    ("Helped PO with backlog prioritization", 
     "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Improved data quality checks in Spark pipelines", 
     "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Facilitated daily standups and retrospectives", 
     "Animer les cérémonies Scrum (daily, rétrospective, sprint planning)"),
    ("Protected team focus from outside distractions", 
     "Protéger l’équipe des interférences externes"),
    ("Worked with BI and data science to align reporting", 
     "Travailler en collaboration avec les équipes Data Science, BI et IT"),



    ("Implémentation et automatisation sur Azure Data Factory (ADF)",
     "Concevoir et développer des pipelines d'ingestion de données robustes et optimisés à l'aide d'Azure Data Factory"),

    ("Utilisation de Synapse serverless et création de workspace",
     "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),

    ("Création de modèles de données en étoile avec gestion des SCD2",
     "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),

    ("Mise en place de Snowflake, gestion des droits, objets, dynamic tables",
     "Contribuer à l’élaboration des principes d’architecture data et de gouvernance des données"),

    ("Utilisation de Azure DevOps pour gérer les environnements",
     "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),

    ("Réalisation d’analyses de performance sur Snowflake",
     "Mettre en œuvre des processus de validation de la qualité des données"),

    ("Facilitation des cérémonies Agile (Daily, Rétro, Planning)",
     "Faciliter l’ensemble des cérémonies Scrum"),

    ("Accompagnement des équipes pour adopter les pratiques Agile",
     "Agir en tant que coach Agile"),

    ("Soutien au Product Owner dans la priorisation du backlog",
     "Aider le Product Owner à gérer efficacement le backlog produit"),

    ("Analyse des KPIs projet et amélioration continue",
     "Mesurer et analyser la performance de l’équipe via des indicateurs clés"),

    ("Création de rapports Power BI à partir de sources SQL Server",
     "Développer des tableaux de bord et rapports dans Power BI"),

    ("Écriture de requêtes SQL complexes pour extraire des indicateurs clés",
     "Rédiger du code SQL performant"),

    ("Intégration de données depuis des fichiers Excel et des systèmes externes",
     "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),

    ("Collaboration avec les analystes pour définir des rapports pertinents",
     "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
]

negative_pairs = [

    ("Built dashboards in Power BI", "Préparer des repas en cuisine"),
    ("Created REST APIs", "Aider les patients dans les soins"),

   
    ("Collaborated with ML and BI teams", "Mesurer la performance de l’équipe via des burndown charts"),
    ("Helped define data architecture", "Créer des affiches pour un événement social"),
    ("Developed PySpark jobs", "Animer un atelier marketing"),
    ("Implemented Kafka streaming", "Créer du contenu pour les réseaux sociaux"),
    ("Automated deployment with Azure DevOps", "Gérer les présences dans un club de sport"),
    ("Documented pipelines", "Traduire un document marketing"),
    ("Designed KPI dashboards", "Agir en tant que coach Agile (Scrum Master)"),
    ("Helped organize hackathon logistics", "Construire des pipelines avec Apache Spark"),
    ("Création de dashboards dans Power BI",
     "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Faciliter les cérémonies Scrum",
     "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),
    ("Accompagnement du Product Owner",
     "Implémenter des flux de données streaming et batch via Databricks Streaming et Apache Kafka"),
    ("Élaboration de reporting mensuel",
     "Documenter les solutions développées selon les normes internes"),
    ("Écriture de requêtes SQL pour KPIs marketing",
     "Contribuer à l’élaboration des principes d’architecture data et de gouvernance des données"),
]

# Préparation des exemples
train_examples = [
    InputExample(texts=[t1, t2], label=1.0) for t1, t2 in positive_pairs
] + [
    InputExample(texts=[t1, t2], label=0.0) for t1, t2 in negative_pairs
]

# Improved training setup
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)  # Increased from 4

# Use ContrastiveLoss instead of CosineSimilarityLoss for binary labels
# ContrastiveLoss is designed for positive/negative pairs with margin-based learning
train_loss = losses.ContrastiveLoss(
    model,
    distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE,
    margin=0.5  # Margin between positive and negative pairs
)

print(f"Training with {len(train_examples)} examples")
print(f"Positive pairs: {len(positive_pairs)}, Negative pairs: {len(negative_pairs)}")
print(f"Using ContrastiveLoss with margin=0.5")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,  # Increased from 3
    warmup_steps=100,  # Increased from 10
    optimizer_params={'lr': 2e-5},  # Explicit learning rate
    show_progress_bar=True
)

# Sauvegarde du modèle
model.save("modelfinetuned2")
print("✅ Model saved to modelfinetuned2")

The tokenizer you are loading from 'modelfinetuned' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Training with 43 examples
Positive pairs: 28, Negative pairs: 15
Using ContrastiveLoss with margin=0.5


Step,Training Loss


✅ Model saved to modelfinetuned2


In [25]:
from sentence_transformers import SentenceTransformer, util

#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('modelfinetuned1')
model = SentenceTransformer("modelfinetuned2")




def compute_similarity(text1, text2):

    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores.item()


text1 = "Created various Customized and interactive Reports and Dashboards in Power BI."
text2 = "developement de dashboard sur Power BI"
text3 = "Mettre en place un flux de données depuis un ERP vers une database Snowflake"
text4 = "Automatiser les déploiements et les tests des pipelines via Azure DevOps (CI/CD, versioning, rollback)."
text5 = "implémenter un pipeline d’intégration de données entre un système ERP et une base de données Snowflake"
#compute_similarity(text3, text1)
#compute_similarity(text3, text2)
print(compute_similarity(text1, text2))
print(compute_similarity(text1, text5))

The tokenizer you are loading from 'modelfinetuned2' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


0.8477262258529663
0.4947778582572937


In [3]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

def compute_similarity(text1, text2):

    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores.item()


text1 = "Created various Customized and interactive Reports and Dashboards in Power BI."
text2 = "developement de dashboard sur Power BI"
text3 = "potatoes are good"
text4 = "Mettre en place un flux de données depuis un ERP vers une database Snowflake"
text5 = "Automatiser les déploiements et les tests des pipelines via Azure DevOps (CI/CD, versioning, rollback)."
text6 = "implémenter un pipeline d’intégration de données entre un système ERP et une base de données Snowflake"
#compute_similarity(text3, text1)
#compute_similarity(text3, text2)
print(compute_similarity(text1, text6))
print(compute_similarity(text1, text3))
print(compute_similarity(text1, text2))

0.3277907371520996
-0.018768634647130966
0.7804136276245117


now need to do the same but for skills. 

embeddings for skills in same domain : 

# encoding the mission 

In [ ]:
dfmission = pd.read_csv("skills-finder-hackathon-hec-x-sbi/HCK_HEC_XP.csv")

In [ ]:
dfmission = dfmission.drop_duplicates(subset=['USER_ID', 'MISSION_DSC'], keep='first').reset_index(drop=True)

In [ ]:
model = SentenceTransformer("modelfinetuned2")
consultant_embeddings = model.encode(dfmission['MISSION_DSC'].tolist(), convert_to_tensor=True, device='cpu')

The tokenizer you are loading from 'modelfinetuned2' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
